## Alcohol availability in Fayette county’s neighborhoods

This was an exploration aimed at using data from KY's Department of Alcoholic Beverage Control (ABC) to calculate the availability of alcohol in Fayette county’s neighborhoods

I aimed to calculate the rate of liquor licenses per capita (the number of licenses divided by the neighborhood’s population).  I showed the top 20 neighborhoods with the highest rate of alcohol availability, as well as the top 20 neighborhoods with the highest number of licenses. Neighborhoods were defined as US Census Bureau tracts

I utilized two files, downloaded from KY's ABC portal. (https://abc.ky.gov/) One contained all licenses, while the other contained a non-quota subset of licenses 

Population data was taken from the US Cencus Bureau with the hierarchy being country (largest) -> state -> county -> tract -> blockgroup -> block (smallest). I chose to calculate the rate at the cencus tract level  from the 2020 American Communities Survey using the US CEnsus Bureau's API (https://api.census.gov/data/2020/acs/acs5/profile?get=NAME,DP05_0001E&for=tract:*&in=state:21&in=county:067) 

Because the license counts were much smaller than the population counts, I adjusted the rate by multiplying by per 100 or 1000 people (so the rate is X licenses per 1000 people instead).

Each license was associated with a street address, so I utilized geocoding to associate particular addresses with a cencus tract. An API call was made keeping in mind the rate limits. 


#### Parts 9 - 13 contain results 


## My Goals

- For each neighborhood, calculate the rate of liquor licenses per capita (the number of licenses divided by the neighborhood’s population).  
- Show the top 20 neighborhoods with the highest rate of alcohol availability. 
- Show the top 20 neighborhoods with the highest number of licenses.  
- Calculate the rate at the census tract level; tracts typically average 4,000 people by design (with a range of 1,200 and 8,000 people)
- Determine whether or not these two top-20 lists differ and how.  

## Part 1: Test API call for census and geocoding data


In [454]:
import requests
response = requests.get("https://api.census.gov/data/2020/acs/acs5/profile?get=NAME,DP05_0001E&for=tract:*&in=state:21&in=county:067")
geocoding = requests.get(" https://geocoding.geo.census.gov/geocoder/geographies/address?street=789+South+Limestone&city=Lexington&state=KY&benchmark=Public_AR_Census2020&vintage=Census2020_Census2020&layers=6&format=json")


I've made a few simple print statements to test whether my API calls worked

In [455]:
print(response.json()[0])
print(geocoding.json()["result"]["input"]["address"])

['NAME', 'DP05_0001E', 'state', 'county', 'tract']
{'city': 'Lexington', 'street': '789 South Limestone', 'state': 'KY'}


## Part 2: Census API response & Data Cleaning

I first processed the census data from the API response and parsed it into a structured format for easier analysis. the JSON response is split into headers and data, which I used to create a pandas DataFrame. The NAME column, which contains location information, is split into three columns—"Census Tract," "County," and "State" to help me organize my approach. I inferred that the field DP05_0001E represents population totals, so I renamed to "population" for clarity. I standardized the column headers for "state," "county," and "tract" to uppercase to match corresponding fields in the geocoding JSON response. I rearragned the columns and placed "Census Tract" at the start for easier tracking. The original NAME column is retained as a final renaming to ensure compatibility with geocoding data. The processed census data is saved as a CSV file, "census_output.csv," for future reference.

In [456]:
import json
import csv
import pandas as pd

# Capture the headers from the api response
jsondata = response.json()[1:]
# Capture the following rows from the api response
columns = response.json()[0]

# Convert the json data to a pandas dataframe
census_data = pd.DataFrame(jsondata, columns=columns)

# The NAME column is split into 3 columns, "Cnensus Tract", "County", and "State" (in hindsight this is not necessary but helped me organize my approach)
census_data[['census_tract', 'county_name', 'state_name']] = census_data['NAME'].str.split(',', n=2, expand=True)
# After looking at the Population totals for Fayette County’s census tracts, I inferred that DP05_0001E is representative of the tract's population. 
# For the sake of organization and consistency, I renamed state, county, and tract, as uppercase headers. This helped me quickly recognize that there
# were corresponding fields in the geocoding json response
census_data.rename(columns={'state': 'STATE', 'county': 'COUNTY', 'tract':'TRACT', 'DP05_0001E':'population'}, inplace=True)

# I chose to reorganize the columns in a format that was easier for me to keep track of
cols = census_data.columns.tolist()
cols.insert(0, cols.pop(cols.index('census_tract')))
cols.pop(cols.index('NAME'))

census_data = census_data[cols]
# I chose to keep the maintain the census_tract column as "NAME" to keep it consistent with the geocoding json response  
census_data.rename(columns={'census_tract':'NAME'}, inplace=True)
# The census responseis saved as a csv file for later reference and use
census_data.to_csv('census_output.csv', index=False)
census_data

,NAME,population,STATE,COUNTY,TRACT,county_name,state_name
0,Census Tract 1.01,3171,21,067,000101,Fayette County,Kentucky
1,Census Tract 1.02,1584,21,067,000102,Fayette County,Kentucky
2,Census Tract 2,3696,21,067,000200,Fayette County,Kentucky
3,Census Tract 3,2472,21,067,000300,Fayette County,Kentucky
4,Census Tract 4,1983,21,067,000400,Fayette County,Kentucky
...,...,...,...,...,...,...,...
77,Census Tract 42.07,3531,21,067,004207,Fayette County,Kentucky
78,Census Tract 42.08,7671,21,067,004208,Fayette County,Kentucky
79,Census Tract 42.09,4225,21,067,004209,Fayette County,Kentucky
80,Census Tract 42.10,5443,21,067,004210,Fayette County,Kentucky


## Part 3: Import CSVs & Data Cleaning 

I cleaned and formatted the data from the provided CSV files containing active license information. I skipping the initial three rows of ActiveLicenses.csv to ensure correct formatting. The "PremisesCityState" column is split into "premise," "state," and "zip" fields. The "PremisesStreet" and "City" columns are renamed to "street" and "city," respectively. Columns are reordered so that "state" and "zip" replace the original "PremisesCityState" position. I removed the redundant "premise" column as it was a duplicate of the "street" information. The cleaned data is saved as cleaned_licenses.csv.

The same cleaning steps are applied to ActiveLicensesNQ.csv

In [473]:
import pandas as pd

# I saved the active licenses to a csv file, and skipped the first 3 rows, as their inclusion disrupted the formatting 
active_licenses = pd.read_csv('ActiveLicenses.csv', skiprows=3, encoding='utf-8')
# separated the "Premise/City/State" column into separate fields
active_licenses[['premise', 'state', 'zip']] = active_licenses['PremisesCityState'].str.split(' ', n=2, expand=True)
# I converted the "PremisesStreet" and city to shorter lowercase names 
active_licenses.rename(columns={'PremisesStreet':'street', 'City':'city'}, inplace=True)

# I reordered the columns so that the separated columns would take the place of the "PremiseCityState" columns
cols = active_licenses.columns.tolist()
premises_idx = cols.index('PremisesCityState')
cols.insert(premises_idx + 1, cols.pop(cols.index('state')))
cols.insert(premises_idx + 2, cols.pop(cols.index('zip')))
cols.pop(cols.index('PremisesCityState'))

# Since the premise contains the same information as the "street" column (formerly "PremisesStreet" column), I removed the duplicate column
cols.pop(cols.index('premise'))
active_licenses = active_licenses[cols]

# The cleaned and reformatted licenses dataset is saved to a csv file for later use and reference
active_licenses.to_csv('cleaned_licenses.csv', index=False)

#The same cleaning process is applied to the Active LicenseNQ dataset
active_licensesNQ = pd.read_csv('ActiveLicensesNQ.csv', skiprows=3, encoding='utf-8')
active_licensesNQ[['premise', 'state', 'zip']] = active_licensesNQ['PremisesCityState'].str.split(' ', n=2, expand=True)
active_licensesNQ.rename(columns={'PremisesStreet':'street', 'City':'city'}, inplace=True)
cols = active_licensesNQ.columns.tolist()
premises_idx = cols.index('PremisesCityState')
cols.insert(premises_idx + 1, cols.pop(cols.index('state')))
cols.insert(premises_idx + 2, cols.pop(cols.index('zip')))
cols.pop(cols.index('PremisesCityState'))
cols.pop(cols.index('premise'))

active_licensesNQ = active_licenses[cols]
# The cleaned and reformatted non-quota licenses dataset is saved to a csv file for later use and reference
active_licensesNQ.to_csv('cleaned_licensesNQ.csv', index=False)
active_licenses

,SiteID,County,city,DBA,Licensee,street,state,zip,LicenseNumber,LicenseType,Status,IssueDate,EffectiveDate,ExpiryDate
0,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,KY,40507,034-CL-422,Caterer's License,Active,02/24/2016,02/24/2016,11/30/2023
1,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,KY,40507,034-HI-29,Hotel In-Room License,Active,01/29/2016,01/29/2016,11/30/2023
2,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,KY,40507,034-LP-2223,Quota Retail Package License,Active,04/29/2016,04/29/2016,11/30/2023
3,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,KY,40507,034-NQ2-2920,NQ2 Retail Drink License,Active,01/29/2016,01/29/2016,11/30/2023
4,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,KY,40507,034-RS-4300,Special Sunday Retail Drink License,Active,01/29/2016,01/29/2016,11/30/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,27204.0,Fayette,Lexington,Zim's Cafe,"Zim's Cafe , LLC",215 W. Main Street,KY,40507,034-NQ2-3843,NQ2 Retail Drink License,Active,11/15/2018,11/15/2018,11/30/2023
1848,27204.0,Fayette,Lexington,Zim's Cafe,"Zim's Cafe , LLC",215 W. Main Street,KY,40507,034-RS-5498,Special Sunday Retail Drink License,Active,11/15/2018,11/15/2018,11/30/2023
1849,27204.0,Fayette,Lexington,Zim's Cafe,"Zim's Cafe , LLC",215 W. Main Street,KY,40507,034-SP-196439,Sampling License,Active,04/25/2023,04/25/2023,11/30/2023
1850,35579.0,Fayette,Lexington,Zundo,ZUNDO RAMEN AND POKE LLC,127 W Tiverton Way,KY,40503,034-NQ2-193247,NQ2 Retail Drink License,Active,10/13/2022,10/13/2022,11/30/2023


In [458]:
# A geocoding test that I frequently referenced as I was verifying fields across datasets
test = requests.get(" https://geocoding.geo.census.gov/geocoder/geographies/address?street=4089+Iron+Works+Pkwy&city=Lexington&state=KY&benchmark=Public_AR_Census2020&vintage=Census2020_Census2020&layers=6&format=json")
test.json()['result']['addressMatches'][0]['geographies']
test_tract = test.json()['result']['addressMatches'][0]['geographies']['Census Tracts'][0]['NAME']
test_tract

'Census Tract 38.02'

## Part 4: Check for missing SiteIDs

Missing site IDs are identified and grouped by street. The number of missing `SiteID` values (`NaN`s) is calculated for each street. A dataframe is displayed and defined to visualize missing counts by street. Only streets with at least one missing `SiteID` are shown in the filtered output, `nan_count_filtered`.

In [459]:
import numpy as np
# Since the geocoding API response requires a unique ID, I assumed that SiteID would be requried. 
# Therefore, I counted the number of missing IDs and grouped them by their corresponding street.
nan_count = active_licenses.groupby('street')['SiteID'].apply(lambda x: x.isna().sum()).reset_index()
# I created a dataframe to display these nan values and visually see what streets are missing SiteIDs
nan_count.columns = ['street', 'nan_SiteID_count']
# Only nan values with occurences greater than 0 are displayed 
nan_count_filtered = nan_count[nan_count['nan_SiteID_count'] > 0]

nan_count_filtered

,street,nan_SiteID_count
113,125 Barr St,1
222,1685 Jaggie Fox Way,1
325,215 W Main St Ste 250,3
374,2339 Sandersville Rd,1
404,251 W Main St,2
405,251 W Second St,1
512,3333 Bowman Mill Rd,1
531,3415 Entertainment Ct,1
630,4089 Iron Works Pkwy,1
651,430 West Vine St,1


### Determine if an address has non-NaN SiteIDs 

I created a function to check for missing (`NaN`) and non-missing (`non-NaN`) `SiteID` values for each street in the `active_licenses` dataset. It reads the `cleaned_licenses.csv` file to display the `SiteID` status for each street. The number of missing `SiteID`s per street are stored in a DataFrame (`nan_count`). The non-missing values (`non-NaN`) are counted per street, and stored in another DataFrame (`not_nan_count`). Both DataFrames are merged to allow for a side-by-side view of missing vs. non-missing `SiteID` values by street. The function returns only streets with missing `SiteID`s, providing a both `NaN` and non-`NaN` counts for those streets.

In [460]:
import numpy as np
import pandas as pd

# To determine if there were existing site IDs I could use, I captured the number of nan and non-nan values for each address
active_licenses = pd.read_csv('cleaned_licenses.csv', encoding='utf-8')

# A function for checking a streets nan values and corresponding non-nan values 
def check_nan_values(active_licenses):
    # The number of nan Site IDs is counted 
    nan_count = active_licenses.groupby('street')['SiteID'].apply(lambda x: x.isna().sum()).reset_index()
    nan_count.columns = ['street', 'nan_SiteID_count']
    
    # The number of non-nan Site IDs is counted
    not_nan_count = active_licenses.groupby('street')['SiteID'].apply(lambda x: x.notna().sum()).reset_index()
    not_nan_count.columns = ['street', 'not_nan_SiteID_count']
    
    # To visually see the nan vs non-nan occurences for each affected street, I merged the dataframes 
    count_summary = pd.merge(nan_count, not_nan_count, on='street')
    
    # Only the streets with nan values are displayed, resulting in a way to see both nan and non-nan values per street
    nan_count_filtered = count_summary[count_summary['nan_SiteID_count'] > 0]
    return nan_count_filtered

check_nan_values(active_licenses)

,street,nan_SiteID_count,not_nan_SiteID_count
113,125 Barr St,1,0
222,1685 Jaggie Fox Way,1,0
325,215 W Main St Ste 250,3,0
374,2339 Sandersville Rd,1,0
404,251 W Main St,2,0
405,251 W Second St,1,0
512,3333 Bowman Mill Rd,1,0
531,3415 Entertainment Ct,1,0
630,4089 Iron Works Pkwy,1,1
651,430 West Vine St,1,0


### Replace missing SiteIDs

Unique IDs are assigned to missing `SiteID` values in the `active_licenses` dataset. If any street has both missing and existing IDs, the existing ID is used for all entries. A `generate_unique_id` function produces random IDs for streets with no existing `SiteID`s. Duplicates are avoided by checking against a list of `unique_ids`, which stores all IDs in use.

The code iterates over each street in the dataset. A unique ID is generated and assigned to all missing entries for for a particular street. For streets with existing `SiteID`s, the first non-missing ID found is used to fill any missing values. The updated DataFrame is saved to `active_licenses_IDs.csv`.


In [481]:
import pandas as pd
import numpy as np
import random

# IDs are assigned to missing SiteID fields, while any street that has more than one entry with both missing and existing values 
# will be automatically assigned the existing SiteID

random.seed(43)

# Whenever an ID is found to be missing, this function will be called to generate a random ID that is not already in unique_ids
def generate_unique_id(existing_ids):
    while True:
        new_id = random.randint(1000, 99999) 
        if new_id not in existing_ids:
            return new_id

# A list is initialized to contain all unique IDs in the dataset
unique_ids = active_licenses['SiteID'].dropna().unique().tolist()

# A dictionary us initialized to store the assigned SiteID for each street with nan values
street_ids = {}

# each street in the dataset is iterated over
for street in active_licenses['street']:
    # If there are any non-nan values for the current street, they will be stored in a variable called "non_nan_values" 
    non_nan_values = active_licenses.loc[active_licenses['street'] == street, 'SiteID'].dropna().unique()
    
    # If there are no non-nan values, nan values will be replaced with a randomly generated ID
    if non_nan_values.size == 0:
        # all nan IDs for the current street will be stored in a variable called "nan_indices"
        nan_indices = active_licenses.index[(active_licenses['street'] == street) & (active_licenses['SiteID'].isna())]

        # If an ID has not already been assigned to the current street, the "generate_unique_id" function will be called
        if street not in street_ids:
            street_ids[street] = generate_unique_id(unique_ids)
            # The new ID will be added to the unique_ids list
            unique_ids.append(street_ids[street]) 
            
        # The stored ID is assigned to each nan SiteID for the current street
        for index in nan_indices:
            active_licenses.at[index, 'SiteID'] = street_ids[street]

    else:
        # If there are non-nan values for a street, the first non-nan value is used to replace the nan value
        replacement_value = non_nan_values[0]
        active_licenses.loc[(active_licenses['street'] == street) & (active_licenses['SiteID'].isna()), 'SiteID'] = replacement_value


# Save the updated DataFrame to a CSV file
active_licenses.to_csv("active_licenses_IDs.csv", index=False)


# verification to check if the updated dataset still contains null IDs
check_nan_values(active_licenses)

,street,nan_SiteID_count,not_nan_SiteID_count


### verify that generated IDs are consistent for each instance of a street

In [482]:
# verify that generated IDs are consistent for each instance of a street
active_licenses.loc[active_licenses['street'].str.contains('215 W Main St Ste 250', na=False)]

,SiteID,County,city,DBA,Licensee,street,state,zip,LicenseNumber,LicenseType,Status,IssueDate,EffectiveDate,ExpiryDate
418,19865.0,Fayette,Lexington,December 2023,BREEDERS' CUP LIMITED,215 W Main St Ste 250,KY,40507,034-TA-196406,Special Temporary Alcoholic Beverage Auction L...,Active,04/24/2023,12/01/2023,12/30/2023
1143,19865.0,Fayette,Lexington,November 2023,BREEDERS' CUP LIMITED,215 W Main St Ste 250,KY,40507,034-TA-196405,Special Temporary Alcoholic Beverage Auction L...,Active,04/24/2023,11/01/2023,11/30/2023
1151,19865.0,Fayette,Lexington,October 2023,BREEDERS' CUP LIMITED,215 W Main St Ste 250,KY,40507,034-TA-196404,Special Temporary Alcoholic Beverage Auction L...,Active,04/24/2023,10/01/2023,10/30/2023


### verify that NaN values for a street, are replaced by non-NaN values found in other instances of the street

In [471]:

# verify that non-nan values for a street, are replaced by the first non-nan value found for other instances of a street
active_licenses.loc[active_licenses['street'].str.contains('4089 Iron Works Pkwy', na=False)]


,SiteID,County,city,DBA,Licensee,street,state,zip,LicenseNumber,LicenseType,Status,IssueDate,EffectiveDate,ExpiryDate
13,18700.0,Fayette,Lexington,Adult Comedy Night Fundraiser,"UNEXPECTED BLESSINGS FOUNDATION, INC.",4089 Iron Works Pkwy,KY,40511,034-TL-199387,Special Temporary License,Active,10/03/2023,10/20/2023,10/20/2023
816,18700.0,Fayette,Lexington,Kentucky Horse Park,KENTUCKY HORSE PARK,4089 Iron Works Pkwy,KY,40511,034-LP-186334,Quota Retail Package License,Active,09/17/2021,09/17/2021,11/30/2023


## Part 6: Save addresses in batch compatible format

Address data is prepared for a batch API request by creating a CSV file with the required columns: Unique ID (`SiteID`), street address, city, state, and ZIP code. Columns from the `active_licenses` DataFrame are selected and ordered with the necessary formatting to ensure a successful API submission.

In [480]:
import pandas as pd

# A batch API request requires a CSV format where columns are:Unique ID, Street address, City, State, ZIP
# Therefore, I created a dataframe composed of the appropriate columns, and wrote the addresses to a csv file for submission in a batch API request

selected_columns = active_licenses[['SiteID', 'street', 'city', 'state', 'zip']]
selected_columns.to_csv('addresses.csv', index=False,  quoting=1)

print("Addresses saved successfully.")
selected_columns.head()

Addresses saved successfully.


,SiteID,street,city,state,zip
0,22790.0,167 W Main St,Lexington,KY,40507
1,22790.0,167 W Main St,Lexington,KY,40507
2,22790.0,167 W Main St,Lexington,KY,40507
3,22790.0,167 W Main St,Lexington,KY,40507
4,22790.0,167 W Main St,Lexington,KY,40507


## Part 7: Submit API request

`curl` is used to handle the API submission. It posts the `addresses.csv` file to the geocoding API, using parameters `benchmark=4` and `vintage=4` for specific data settings, and outputs the result to `all_geocodes.csv`. I considered batch processing, but a bug affected only the second batch. I found that it was more efficient to submit all addresses in a single request since it stayed within the 10,000 limit.

In [463]:
# I initially submit a batch request using strictly python, but found that there were some formatting issues. I could have
# fixed those issues, but chose to use curl instead since it largely formatted the response correctly. I also considered processing 
# the addresses in batches, but ran into formatting issues for only batch 2. It was a very odd bug, so I chose instead to just submit all addresses
# at once since they did not exceed the limit of 10000, and all followed the same format
!curl --form addressFile=@addresses.csv --form benchmark=4 --form vintage=4 \
https://geocoding.geo.census.gov/geocoder/geographies/addressbatch --output all_geocodes.csv



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  104k    0     0  100  104k      0  83603  0:00:01  0:00:01 --:--:-- 83680
100  104k    0     0  100  104k      0  46997  0:00:02  0:00:02 --:--:-- 47026
100  104k    0     0  100  104k      0  32575  0:00:03  0:00:03 --:--:-- 32583
100  104k    0     0  100  104k      0  24981  0:00:04  0:00:04 --:--:-- 24985
100  104k    0     0  100  104k      0  20226  0:00:05  0:00:05 --:--:-- 20551
100  104k    0     0  100  104k      0  17001  0:00:06  0:00:06 --:--:--     0
100  104k    0     0  100  104k      0  14675  0:00:07  0:00:07 --:--:--     0
100  104k    0     0  100  104k      0  12895  0:00:08  0:00:08 --:--:--     0
100  104k    0     0  100  104k      0  11506  0:00

## Part 8: Add headers to the geocodes

The raw geocoding results are read from `all_geocodes.csv`. The batch API response does not include headers, so I inferred custom headers using the JSON geocodes response, and saved the data with headers as `all_geocodes_with_headers.csv`. Only columns that will be used to identify census information will be defined, with `"STATE"`, `"COUNTY"`, and `"TRACT"`. I used uppercase for consistency. Unnecessary or ambiguous columns were left unnamed.

In [2]:
import pandas as pd

# The batch API call did not include headers. Based on the json geocodes, I was able to infer what important columns represented. 
# Since I only needed tract information, I did not worry about defining ambiguous columns, as they were not necessary for the results. 
# Coulmns that will be used to identify population in the cencus dataset are formatted in the same manner with "state", "county", and "tract" 
# columns defined in uppercase
headers = [
    "??", "Original Address", "Match Status", "Match Type", 
    "Matched Address", "Longitude/Latitude", "TigerLineId", "Side of Street", 
   "STATE" , "COUNTY", "TRACT", "??"
]


# The previously created geocodes CSV is read as a data frame
all_geocodes = pd.read_csv("all_geocodes.csv", header=None)

# The custom headers are added to the all_geocodes dataframe
all_geocodes.columns = headers
all_geocodes.columns = headers


# A new csv file with the geocodes and headers is saved
all_geocodes.to_csv("all_geocodes_with_headers.csv", index=False)


print("Headers added successfully. Saved as all_geocodes_headers.csv")
all_geocodes.head()

Headers added successfully. Saved as all_geocodes_headers.csv


,??,Original Address,Match Status,Match Type,Matched Address,Longitude/Latitude,TigerLineId,Side of Street,STATE,COUNTY,TRACT,??
0,488.0,"1116 Winburn Dr, Lexington, KY, 40511",Match,Exact,"1116 WINBURN DR, LEXINGTON, KY, 40511","-84.47598578654924,38.089438386187396",31945598.0,R,21.0,67.0,3804.0,2003.0
1,1833.0,"3101 Richmond Rd, Lexington, KY, 40509",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9744.0,"2688 Pink Pigeon Pkwy, Lexington, KY, 40509",Match,Exact,"2688 PINK PIGEON PKWY, LEXINGTON, KY, 40509","-84.42639655840867,38.02245148652963",31954114.0,R,21.0,67.0,3908.0,4040.0
3,29912.0,"222 Rosemont Gdn, Lexington, KY, 40503",Match,Exact,"222 ROSEMONT GARDEN, LEXINGTON, KY, 40503","-84.5221953519258,38.019400865010994",31922565.0,L,21.0,67.0,2800.0,1000.0
4,1121.0,"2300 Versailles Rd, Lexington, KY, 40504",Match,Exact,"2300 VERSAILLES RD, LEXINGTON, KY, 40504","-84.55198240375927,38.046673831061355",31951790.0,R,21.0,67.0,3000.0,5011.0


### determine if an api call  was successful when using imputed Site IDs

In [3]:
# Check to determine if an api call using addresses with imputed Site IDs was successful
all_geocodes.loc[all_geocodes['Original Address'].str.contains('2339 Sandersville Rd', na=False)]

,??,Original Address,Match Status,Match Type,Matched Address,Longitude/Latitude,TigerLineId,Side of Street,STATE,COUNTY,TRACT,??
52,6053.0,"2339 Sandersville Rd, Lexington, KY, 40511",Match,Exact,"2339 SANDERSVILLE RD, LEXINGTON, KY, 40511","-84.52537550469839,38.08680164496241",636246750.0,R,21.0,67.0,3704.0,2031.0


## Part 9: Merge Datasets

Geocoding data is processed and merged with license information. Only essential columns, are retained including `Original Address`, `STATE`, `COUNTY`, and `TRACT`. It separates the `Original Address` into `street`, `city`, `state`, and `zip` columns while ensuring compatibility with the license data by parsing in reverse (starting with zip and ending with street) limiting the split to the last three commas. Rows with missing `TRACT` values are removed to reduce the chances of potential result skewing. Both the geocoding and license datasets are standardized to lowercase and whitespace-trimmed to ensure consistency for merging. The geocoding information is merged with the license data using shared address columns. Each address is associated with the correct license by merging on shared address columns. The combined dataset is saved as `licenses_w_tract.csv`.

In [4]:
# the geocode dataset with headers is loaded from a csv file.
geo_w_headers = pd.read_csv('all_geocodes_with_headers.csv', encoding='utf-8')
# Only pertinent geocode information will be used. There is no need to include fields like "Longitude/Latitude", "TigerLineId", etc. 
# Additionally, there is no need to identify additional columns since we are trying to associate geocoding information with the original addresses. 
# "Original Address" column is kept as reference.
select_geo_info = geo_w_headers[['Original Address', 'STATE', 'COUNTY', 'TRACT']].copy()

# The original address is split into separate columns for compatibility with the license data. Because there are premises that include commas,
# the "Original Address" column is separated starting with the zip code and ending with the address. There are only 3 necessary columns that I 
# will used to identify the appropriate corresponding license. Therefore, I stopped splitting the address after 3 commas
select_geo_info[['street', 'city', 'state', 'zip']] = (
    select_geo_info['Original Address']
    .apply(lambda x: pd.Series(x.rsplit(',', 3)))
)

# I dropped rows with nan values in the 'TRACT' column, because that indicates that there was no information found in the api. It is not as easy to 
# be sure that an address/license is associated with the appropriate tract, so I removed values that could skew the results (albeit slightly) 
select_geo_info = select_geo_info.dropna(subset=['TRACT'])
select_geo_info['TRACT'] = pd.to_numeric(select_geo_info['TRACT'], errors='coerce').astype(int)

# The license information with imputed SiteIDs is loded 
license_info = pd.read_csv('active_licenses_IDs.csv', encoding='utf-8')
# A dataframe is defined to contain relevant columns. 'street','city','state', and 'zip' will be used to identify the appropriate license/address
select_license_info = license_info[['street', 'city', 'state', 'zip', 'LicenseType', 'Status', 'EffectiveDate']].copy()

# the formatting of street, city, state, and zip are standardized to match between dataframes
select_geo_info[['street', 'city', 'state', 'zip']] = select_geo_info[['street', 'city', 'state', 'zip']].apply(lambda x: x.str.strip().str.lower())
select_license_info[['street', 'city', 'state', 'zip']] = select_license_info[['street', 'city', 'state', 'zip']].apply(lambda x: x.str.strip().str.lower())

# the dataframes are merged without removing duplicates to keep all addresses intact even if they share a tract
merged_data = select_geo_info.merge(select_license_info, on=['street', 'city', 'state', 'zip'], how='left')

# the combined data is saved as a CSV
merged_data.to_csv("licenses_w_tract.csv", index=False)

merged_data.head()

,Original Address,STATE,COUNTY,TRACT,street,city,state,zip,LicenseType,Status,EffectiveDate
0,"1116 Winburn Dr, Lexington, KY, 40511",21.0,67.0,3804,1116 winburn dr,lexington,ky,40511,NQ Retail Malt Beverage Package License,Active,10/08/2020
1,"2688 Pink Pigeon Pkwy, Lexington, KY, 40509",21.0,67.0,3908,2688 pink pigeon pkwy,lexington,ky,40509,NQ Retail Malt Beverage Package License,Active,10/11/2018
2,"2688 Pink Pigeon Pkwy, Lexington, KY, 40509",21.0,67.0,3908,2688 pink pigeon pkwy,lexington,ky,40509,NQ4 Retail Malt Beverage Drink License,Active,05/27/2015
3,"222 Rosemont Gdn, Lexington, KY, 40503",21.0,67.0,2800,222 rosemont gdn,lexington,ky,40503,NQ2 Retail Drink License,Active,05/03/2021
4,"222 Rosemont Gdn, Lexington, KY, 40503",21.0,67.0,2800,222 rosemont gdn,lexington,ky,40503,Special Sunday Retail Drink License,Active,05/03/2021


## Part 10: Calculate the rate of liquor licenses per capita 

The number of licenses per tract are calculated by grouping and counting rows based on the `TRACT` column. Census data is read into a DataFrame and retains only the relevant columns: `NAME`, `population`, and `TRACT`. The license counts are merged with the census information, resulting in a new DataFrame that includes both the number of licenses and the population for each tract. The rate of licenses per tract is computed by dividing the license count by the population and multiplying by 1,000, then rounding the result to two decimal places. The dataset is sorted first by the number of licenses per tract and saved as `licenses_per_tract_ordered.csv`. THe dataset is then sorted by the calculated rate and saved as `rate_per_tract_ordered.csv`.

In [6]:
# the number of licenses per tract is calculated by grouping and counting rows by tract
licenses_per_tract = select_geo_info.groupby('TRACT').size().reset_index(name='license_count')
licenses_per_tract

# THe census information is read into a dataframe
census_info = pd.read_csv('census_output.csv', encoding='utf-8')
# Only the "NAME", "population", and "tract" columns are used
census_info = census_info[['NAME','population', 'TRACT']].copy()
census_info

# To view the number of licenses per tract, I have merged each tract's license count into a dataframe containing tract information
license_count_per_tract = census_info.merge(licenses_per_tract, on=['TRACT'], how='right')
# The rate per tract is calculated and saved into its own column
license_count_per_tract['rate'] = (license_count_per_tract['license_count'] / license_count_per_tract['population'] * 1000).round(2)

# The dataset is first ordered by number of licenses per tract. The results are saved into a csv file.
ordered_by_count = license_count_per_tract.sort_values(by='license_count', ascending=False)
ordered_by_count.to_csv("licenses_per_tract_ordered.csv",index=False)

# The dataset is ordered by the rate. The results are saved into a csv file
ordered_by_rate = ordered_by_count.sort_values(by='rate', ascending=False)
ordered_by_rate.to_csv("rate_per_tract_ordered.csv",index=False)

## Part 11: Show the top 20 neighborhoods with the highest number of licenses.

In [468]:
ordered_by_count.head(20)

,NAME,population,TRACT,license_count,rate
0,Census Tract 1.01,3171,101,76,23.97
71,Census Tract 42.04,5577,4204,50,8.97
11,Census Tract 10,1598,1000,35,21.90
10,Census Tract 9,5951,900,33,5.55
52,Census Tract 39.08,5758,3908,25,4.34
73,Census Tract 42.08,7671,4208,25,3.26
28,Census Tract 28,4028,2800,24,5.96
7,Census Tract 7,2610,700,21,8.05
1,Census Tract 1.02,1584,102,21,13.26
27,Census Tract 27,3858,2700,20,5.18


## Part 12: Show the top 20 neighborhoods with the highest rate of alcohol availability.

In [469]:
ordered_by_rate.head(20)

,NAME,population,TRACT,license_count,rate
0,Census Tract 1.01,3171,101,76,23.97
11,Census Tract 10,1598,1000,35,21.90
1,Census Tract 1.02,1584,102,21,13.26
71,Census Tract 42.04,5577,4204,50,8.97
7,Census Tract 7,2610,700,21,8.05
13,Census Tract 13,2131,1300,15,7.04
28,Census Tract 28,4028,2800,24,5.96
48,Census Tract 38.02,1926,3802,11,5.71
10,Census Tract 9,5951,900,33,5.55
3,Census Tract 3,2472,300,13,5.26


## Part 13: Discussion on whether or not these two top-20 lists differ and how
There appears to be some overlap between the top 20 census tracts based on license count and rate. There are some differences in ranking but the common entries are tracts that rank high in both license count and rate. There seem to be certain locations that have both a significant number of licenses and a high rate per population. Tracts with high rates despite having fewer licenses appear to have smaller populations that drive the rate up. Likewise, some tracts with high license counts but lower ranking rates have larger populations. Overall a basic yet interesting look at Alchohol availability in Fayette county. In the future I'd like to expand this study and delve deeper into some additional statistics that incorporate rate of crime and domestic violence.